In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('Hybrid.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_ESP_O1        68 non-null     float64
 4   Cate_ESP_H1        68 non-null     float64
 5   Cate_ESP_O2        68 non-null     float64
 6   Cate_ESP_H2        68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  Cate_RDKit_PC_0    68 non-null     float64
 13  Cate_RDKit_PC_1    68 non-null     float64
 14  Cate_RDKit_PC_2    68 non-null     float64
 15  Cate_RDKit_PC_3    68 non-null     float64
 16  Cate_RDKit_PC_4    68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_ESP_O1  Cate_ESP_H1  Cate_ESP_O2  Cate_ESP_H2  \
0      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
1      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
2      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
3      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
4      -5.51       0.22    -0.542028     0.433177    -0.582447     0.440296   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  ...  P_HOMO  \
0         1.363         0.969         1.378         0.965  ...   -7.85   
1         1.363         0.969         1.378         0.965  ...   -8.16   
2         1.363         0.969         1.378         0.965  ...   -7.22   
3         1.363         0.969         1.378         0.965  ...   -7.85   
4         1.364         0.969         1.379         0.965  ...   -7.85   

   P_LUMO   P_ESP_P  P_ESP_=O   P_ESP_H  P_BL_P=O  P_BL_PH  P_

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#SVM_a_X,log(y)
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    
    log_y_train = np.log(y_train)
    
    param_svm={'C':[0.1,1,5,10,50,100,200,300,400,500,750,1000],
               'gamma':[100,10,1,0.1,0.01,0.001,0.0001,0.00001]}
    reg_svm = GridSearchCV(SVR(kernel='rbf', epsilon=0.3), param_grid=param_svm, cv=5, n_jobs=16)
    reg_svm.fit(a_X_train,log_y_train['Yield'])
    reg_best = reg_svm.best_estimator_
    y_pred1 = np.exp(reg_best.predict(a_X_train))
    y_pred2 = np.exp(reg_best.predict(a_X_test))
    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    parametors = reg_svm.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_svm.best_score_
    data_cv_score.append(cv_score)
    
    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_svm.best_params_)
    print("Best Model Score:",reg_svm.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.9259861457815042
R2_test: 0.7731163405187326
----------------------
seed: 1
Best Model Parameter: {'C': 100, 'gamma': 0.001}
Best Model Score: 0.8943977740236135
R2_test: 0.8336368046713827
----------------------
seed: 2
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.9328009334146243
R2_test: 0.8215640202841681
----------------------
seed: 3
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.9006063346911791
R2_test: 0.834002334147416
----------------------
seed: 4
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.9303609049263933
R2_test: 0.8078900961080041
----------------------
seed: 5
Best Model Parameter: {'C': 750, 'gamma': 0.0001}
Best Model Score: 0.8790395800107873
R2_test: 0.7600077212655167
----------------------
seed: 6
Best Model Parameter: {'C': 750, 'gamma': 0.0001}
Best Model Score: 0.878016177967392
R2_te

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
print(data_all)

data_all.to_csv('../../score/Hybrid//SVM_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.881432    9.349669   6.144402  0.773116  17.226506  10.701141   
1     0.853072   11.526837   7.667277  0.833637  12.695695   7.525612   
2     0.857703   11.473392   6.684567  0.821564  13.136153   8.681013   
3     0.841277   12.167294   7.942026  0.834002  12.622028   7.738384   
4     0.893176    9.596508   5.861054  0.807890  14.292485   9.584558   
5     0.838745   11.426912   6.623112  0.760008  17.084484  10.561491   
6     0.841106   11.419287   7.309998  0.774884  16.459590   9.584912   
7     0.895696    9.988310   6.130567  0.824020  12.630423   8.847323   
8     0.852764   11.383540   6.853756  0.761292  15.359023  10.122791   
9     0.860436   11.892843   8.042236  0.823853  11.141058   6.669935   
mean  0.861541   11.022459   6.925899  0.801427  14.264744   9.001716   

      cv_score  
0     0.925986  
1     0.894398  
2     0.932801  
3     0.900606  
4     0.930361  
5     0.879040  
6   